# PyTorch - How to use the GPU
Most of the computations that are done when training your deep learning model will consist of matrix multiplications. GPUs are optimized for these type of computations and can therefore greatly decrease the time it takes to train your model. When your models become larger, this can save hours or even days of training. This lab explains how to use the GPU in PyTorch. 

We recommend you use the GPU provided by Colab instead of the one in your laptop, as these are much better and also simplify the tool installation. Also, PyTorch makes use of CUDA, which is a platform for general purpose computing on GPUs. Not all GPUs are compatible with CUDA. 

PyTorch tensors are either allocated on the CPU or GPU. Tensors located on the CPU cannot interact with those on the GPU and vice versa. This is something you need to keep track of when programming your models. 

In [1]:
import torch

If you are working in Colab, try enabling the GPU by, in the menu above, selecting 'Runtime' and 'Change runtime type'.

In [2]:
if torch.cuda.is_available():
  print('GPU is available!')
  device = torch.device('cuda')
else:
  print('GPU is not available!')
  device = torch.device('cpu')

GPU is available!


Next, we allocate some tensors on both the CPU and GPU.

In [3]:
tensor_1 = torch.randn(4, device=torch.device('cpu'))   # CPU tensor
tensor_2 = torch.randn(4, device=torch.device('cuda'))  # GPU tensor
tensor_3 = torch.randn(4, device=device)                # Initialized on the device being used
tensor_4 = torch.randn(4)                               # By default, tensors are initialized as CPU tensors
tensor_5 = torch.FloatTensor([1,2,3,4])                 # CPU tensor
tensor_6 = torch.cuda.FloatTensor([1,2,3,4])            # GPU tensor


Since the following tensors are not allocated on the same device, they cannot be multiplied:

In [4]:
tensor_1 * tensor_2

RuntimeError: expected device cpu but got device cuda:0

The .to() function can be used to move a tensor from one device to another. However, the usage of this function should be minimized, as it is a relatively expensive operation.

In [ ]:
tensor_3 * tensor_4.to(device)

Since model parameters are tensors, they are allocated on either the CPU or GPU as well. Therefore, models allocated on some device cannot process data on another device. Using the .to() function it is possible to move entire models between devices as well (if they extend torch.nn.Module).

In [ ]:
from torch.nn import Module, Parameter

# Define a small Module example to move between devices
class Neuron(Module):

  def __init__(self):
    super(Neuron, self).__init__()
    self.weights = Parameter(torch.randn(4))
    self.bias = Parameter(torch.randn(1))
  
  def forward(self, x):
    return torch.sigmoid(torch.sum(self.weights * x) + self.bias)


neuron = Neuron().to(device)
neuron.forward(tensor_3)

### Exercise
Try modifying the code below such that the training loop uses the GPU

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# This code does not need to be modified!

class TwoLayerNet(nn.Module):

    def __init__(self, input_size, hidden_size1, output_size):
        super(TwoLayerNet , self).__init__()
        
        self.layer1 = nn.Linear(input_size, hidden_size1, bias=False)
        self.layer2 = nn.Linear(hidden_size1, output_size, bias=False)
        
    def forward(self, x):
        y       = self.layer1(x)
        y_hat   = F.relu(y)
        z       = self.layer2(y_hat)
        return F.softmax(z, dim=1)
  

In [2]:
# This code does not need to be modified!

import torchvision.datasets
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambda x: x.squeeze()),
                                ])

trainset = torchvision.datasets.MNIST(root='./data_mnist',
                                      train=True,
                                      download=True,
                                      transform=transform
                                      )

testset = torchvision.datasets.MNIST(root='./data_mnist',
                                     train=False,
                                     download=True,
                                     transform=transform
                                     )



Using downloaded and verified file: ./data_mnist/MNIST/raw/train-images-idx3-ubyte.gz
Extracting ./data_mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data_mnist/MNIST/raw
Using downloaded and verified file: ./data_mnist/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting ./data_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data_mnist/MNIST/raw
Using downloaded and verified file: ./data_mnist/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data_mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data_mnist/MNIST/raw


EOFError: Compressed file ended before the end-of-stream marker was reached

In [ ]:
# This code does not need to be modified!

bs=128

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=bs,
                                          shuffle=True,
                                          drop_last=True
                                          )

testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=bs,
                                         shuffle=True,
                                         drop_last=True
                                         )

In [ ]:
# Modify the following code such that it uses the GPU

net=TwoLayerNet(784, 64, 10)

criterion = nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(net.parameters() , lr=0.1)

for epoch in range(1, 5):

  for i, (minibatch_data, minibatch_label) in enumerate(trainloader):

      # Set dL/dU, dL/dV, dL/dW to be filled with zeros
      optimizer.zero_grad()
      
      #reshape the minibatch
      inputs = minibatch_data.view(bs, 784)

      # forward the minibatch through the net  
      prob=net(inputs) 
      
      # Compute the average of the losses of the data points in the minibatch
      loss = criterion(prob , minibatch_label) 
      
      # backward pass to compute dL/dU, dL/dV and dL/dW    
      loss.backward()
      
      # do one step of stochastic gradient descent: U=U-lr(dL/dU), V=V-lr(dL/dU), ...
      optimizer.step()

In [ ]:
# This code does not need to be modified!

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# choose a picture at random
im_minibatch, label_minibatch = iter(testloader).next()
im, label = im_minibatch[0].cpu(), label_minibatch[0].cpu()

# Function to show an image tensor
def show(X):
    if X.dim() == 3 and X.size(2) == 3:
        plt.imshow(X.numpy())
        plt.show()
    elif X.dim() == 2:
        plt.imshow(   X.numpy() , cmap='gray'  )
        plt.show()
    else:
        print('WRONG TENSOR SIZE')

# diplay the picture
show(im)

# feed it to the net and display the confidence scores
prob = net.cpu()(im.view(1,784)) 

print('Confidence scores:\n' + '\n'.join(['{}: {}'.format(i, p.item()) for i, p in enumerate(prob.squeeze())]))

print('\nLabel with highest confidence score: {}'.format(torch.argmax(prob).item()))
